Instructions: \\
1.Create a data folder with name "data" by clicking on folder icon on left side of colab notebook
2.Upload COVID-Dialogue-Dataset.txt to the data folder
3. Run the dataset pre-processing cells, will create data.source, data.target files
4. Train the model - will take 7-8 minutes
5. Create a template folder with name "templates" by clicking on folder icon on left side of colab notebook
5. Upload chatroom.html file in the template folder
6. Run flask app hosting code
7. COVID Conversational bot is ready! You can use text or voice to enter your queries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Dataset preprocessing

In [ ]:

!pip install simpletransformers -q
!pip install nltk==3.4.5

import nltk
print(nltk.__version__)

try:
  from nltk.translate.meteor_score import meteor_score
  print('Meteor score will not work without the right ntlk version')
except ImportError:
  print('Still import issue')

     |████████████████████████████████| 225kB 3.0MB/s 
     |████████████████████████████████| 1.2MB 12.4MB/s 
     |████████████████████████████████| 1.8MB 16.3MB/s 
     |████████████████████████████████| 225kB 22.9MB/s 
     |████████████████████████████████| 8.2MB 20.2MB/s 
     |████████████████████████████████| 122kB 42.1MB/s 
     |████████████████████████████████| 2.3MB 35.9MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 3.3MB 38.8MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 174kB 32.4MB/s 
     |████████████████████████████████| 102kB 10.2MB/s 
     |████████████████████████████████| 133kB 37.4MB/s 
     |████████████████████████████████| 245kB 35.1MB/s 
     |████████████████████████████████| 112kB 41.3MB/s 
     |████████████████████████████████| 4.2MB 40.4MB/s 
     |████████████████████████████████| 112kB 44.2MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 

In [ ]:
import tqdm
import spacy
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from simpletransformers.seq2seq import Seq2SeqModel
from sklearn.model_selection import train_test_split
import os
import os.path
import tensorflow as tf
import os
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.nist_score import sentence_nist
import pandas as pd

In [ ]:
import os
src = "/content/data/data.source"
tgt = "/content/data/data.target"
dataset_file = "/content/data/COVID-Dialogue-Dataset-English.txt" ##For a session, add this dataset to local colab directory and change the paths accordingly

In [ ]:

Dialogue_on=False
Patient_on=False
Doctor_on=False
Patient_input=''
Doctor_output=''
Patient_idx=0
Doctor_idx=0
with open(dataset_file) as f, open(src,'w') as source, open(tgt,'w') as target:
    for line in f.readlines():
        line=line.strip()
        if line=='Dialogue':
            Dialogue_on=True
            Patient_on=False
            Doctor_on=False
            if Patient_input!='':
                source.write(Patient_input+'\n')
                Patient_idx+=1
                Patient_input=''
                assert Doctor_output==''
                target.write(' \n')
                Doctor_idx+=1
            elif Doctor_output!='':
                target.write(Doctor_output+'\n')
                Doctor_idx+=1
                if Doctor_idx!=Patient_idx:
                    raise AssertionError(Doctor_output)
                Doctor_output=''
                assert Patient_input==''
        elif line=='Patient:':
            Patient_on=True
            Doctor_on=False
            Patient_input=''
            if Doctor_output!='':
                target.write(Doctor_output+'\n')
                Doctor_idx+=1
                if Doctor_idx!=Patient_idx:
                    raise AssertionError(Doctor_output)
                Doctor_output=''
        elif line=='Doctor:':
            Doctor_on=True
            Patient_on=False
            Doctor_output=''
            if Patient_input!='':
                source.write(Patient_input+'\n')
                Patient_input=''
                Patient_idx+=1
        elif 'id=' in line and len(line)<10:
            Dialogue_on=False
            Patient_on=False
            Doctor_on=False
        else:
            if Dialogue_on and Patient_on:
                Patient_input=Patient_input+line+' '
            elif Dialogue_on and Doctor_on:
                line=line.replace('In brief:','')
                line=line.split()
                for i in range(len(line)):
                    if 'https://' in line[i]:
                        line[i]='https://www.healthtap.com/blog/covid-19-care-guidelines'
                line=' '.join(line)
                Doctor_output=Doctor_output+line+' '
    #for loop ended
    if Patient_input!='':
        source.write(Patient_input+'\n')
        Patient_input=''
        assert Doctor_output==''
        target.write(' \n')
    elif Doctor_output!='':
        target.write(Doctor_output+'\n')
        Doctor_output=''
        assert Patient_input==''

source.close()
target.close()

In [ ]:
with open(src, "r") as src:
  source = src.readlines()
with open(tgt, "r") as tgt:
  target = tgt.readlines()
train_contexted = []
for i in range(len(source)):
  row = []
  row.append(source[i])
  row.append(target[i])

  train_contexted.append(row)
  columns = ['input_text', 'target_text'] 
trn_df = pd.DataFrame.from_records(train_contexted, columns=columns)

In [ ]:
print(trn_df)

                                            input_text                                        target_text
0    Hello doctor, I get a cough for the last few d...  Hello, I understand your concern. I just have ...
1    Thank you doctor, I have phlegm but not a lot....  Hi, I would recommend you take n-acetylcystein...
2    Hello doctor, I am suffering from coughing, th...  Hello, I can understand your concern. In my op...
3    Thank you doctor, After doing all these I can ...  Hi, yes, upload in this query only. I will see...
4    Thank you doctor, As per your guidelines, I ha...  Hi, I can understand your concern. I have gone...
..                                                 ...                                                ...
611  If I get coronavirus, are there ways to avoid ...  Coronavirus, COVID-19 has high affinity to the...
612  I was on a plane on March 8th have been home e...  COVID These are HealthTap https://www.healthta...
613  My son has a fever, I am concerned due to

In [ ]:
train, val  = train_test_split(trn_df, test_size = 0.2)
val, test = train_test_split(val, test_size = 0.5)
print(len(train))
print(len(val))
print(len(test))
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')

492
62
62
Device: cuda


#Using simple transformers to finetune on BART model

In [ ]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 100,
    "train_batch_size": 5, # check if we can have bigger weights. 
    "eval_batch_size": 1,
    "output_dir": 'weights',
    "num_train_epochs": 10,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_during_training": True,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": False,
    "use_multiprocessing": True,
    "gradient_accumulation_steps": 1,
    "max_length": 200,
    "manual_seed": 4,
}


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
)




In [ ]:
model.train_model(train, eval_data=val)

  0%|          | 0/492 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/99 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  0%|          | 0/62 [00:00<?, ?it/s]

...

Generating outputs:   0%|          | 0/62 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/62 [00:00<?, ?it/s]

(990,
 {'eval_loss': [3.196609056765033,
   2.974816217537849,
   2.885557561151443,
   2.8669083868303606,
   2.9171387216737195,
   3.003468395721528,
   3.1336691557399687,
   3.229903613848071,
   3.2876662179347007,
   3.329426118923772],
  'global_step': [99, 198, 297, 396, 495, 594, 693, 792, 891, 990],
  'train_loss': [1.9368821382522583,
   3.9867048263549805,
   2.064481496810913,
   2.2886250019073486,
   1.9452240467071533,
   1.8069230318069458,
   1.5365842580795288,
   1.4923239946365356,
   1.432935357093811,
   0.8690037727355957]})

In [ ]:
def save_prediction(labels, preds):

  for prediction in preds:
    with open('prediction.txt', 'a') as f:
      f.write(prediction + '\n')

  print('complete')



print(model.eval_model(test, save = save_prediction))

  0%|          | 0/62 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/62 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/62 [00:00<?, ?it/s]

complete
{'eval_loss': 3.489054542876059, 'save': None}


In [ ]:
with open('prediction.txt', 'r') as preds:
  predictions = preds.readlines()
test1 = "Can I go for Coronavirus testing if my nose is blocked and I have traveled from a high risk country?"
inference = model.predict([test1])
print(inference)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Yes. Yes. Yes you can. Yes it is a good idea. Would you like to video or text chat with me?  \n']


In [ ]:

test

,input_text,target_text
220,"Will getting either of the pneumonia vaccines,...",No. The vaccines prevent bacterial pneumonia. ...
574,I heard ACE inhibitors might exacerbate corona...,Coronavirus and ACEI. Recent publications incl...
438,On Saturday evening I arrived from a vacation ...,Covid19 HiYou cannot go in and see your doctor...
62,"I ve had symptoms of cough, headache, stuffy n...",Hello and Welcome to ‘Ask A Doctor’ service. I...
179,Yes I had pneumonia and now have Costochondrit...,Do you have diagnosed it correctly ? Go for HR...
...,...,...
350,I have chest congestion and I feel heavy when ...,"? telephonic consult If you are able, consult ..."
483,And i travelled to the UK (back in RSA on the ...,Infection Influenza versus Coronavirus. Typica...
441,How do I know if I have a normal cold or maybe...,Common cold With sin. The corona virus Causes ...
585,I have mild irritation in my chest but I am no...,"Probably. We should all be cautious , not worr..."


In [ ]:
nltk.download('wordnet')


"""
The code crashes when I evaluate within the evalulate function. 
-This is an alternative
"""

def nist_2(labels, preds):

    label = ' '.join([str(elem) for elem in labels])
    prediction = ' '.join([str(elem) for elem in preds])
  
    if len(prediction) < 2 or len(label) < 2:
        return 0
    return sentence_nist([label], prediction, 2)

def nist_4(labels, preds):

    label = ' '.join([str(elem) for elem in labels])
    prediction = ' '.join([str(elem) for elem in preds])

    if len(prediction) < 4 or len(label) < 4:
        return 0

    return sentence_nist([label], prediction, 4)

def calculate_m_score(target, predictions, length):

  score = 0

  for t, p in zip(target, predictions):
    score += meteor_score(t, p)

  
  return score / length


predictions = []

with open('prediction.txt') as fp:
  line = fp.readline()
  line = line.strip()
  
  while line:
    if len(line) > 1:
      predictions.append(line)
    line = fp.readline()

# label = ' '.join([str(elem) for elem in labels])
pred = ' '.join([str(elem) for elem in predictions])
target = test.target_text

bleu2 = sentence_bleu(target, pred, weights=tuple(1 / 2 for i in range(2)))
bleu4 = sentence_bleu(target, pred, weights=tuple(1 / 4 for i in range(2)))
nist2 = nist_2(target, pred)
nist4 = nist_4(target, pred)


print('bleu2 is {}'.format(bleu2))
print('bleu4 is {}'.format(bleu4))
print('nist2 is {}'.format(nist2))
print('nist4 is {}'.format(nist4))

meteor = calculate_m_score(target, predictions, 71)

print('meteor is {}'.format(meteor))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


bleu2 is 0.09631673968589775
bleu4 is 0.31034938325361267
nist2 is 2.9829908521057353
nist4 is 3.20387881675798
meteor is 0.04951303290507413


In [ ]:
!pip install flask-ngrok >> /dev/null

In [ ]:
!pip install emoji

     |████████████████████████████████| 133kB 2.1MB/s 


In [ ]:
!pip install COVID19py

  Created wheel for COVID19py: filename=COVID19Py-0.3.0-cp37-none-any.whl size=4718 sha256=882a6ea492b8ad97a100f3cb10b7d561bbbafd6f103be8e7f19c44d8953d8a36
  Stored in directory: /root/.cache/pip/wheels/23/78/50/78aaca989f422d636b01b9c55cd5e327fef838e973690f886c
Successfully built COVID19py


In [ ]:
from flask import Flask, request, render_template, redirect, url_for
import requests
import json
import os
import emoji
import re
#from static.macro import PROFANITY, GREETINGS, FAREWELL
import COVID19Py
import random

In [ ]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok

In [ ]:
PROFANITY = ['fuck',  'asshole', "bitch", "bullshit",
             "cunt", "motherfucker", "holy shit", 
             "nigga", "crap", "shit", "prick", "goddamn", "fucking", "slut"]


greetings_1 = "hello"
greetings_2 = "Hi there"
greetings_3 = "Hi"
greetings_4 = "Hello"
greetings_5 = "hi"
greetings_6 = "hey"
greetings_7 = "Hey"

GREETINGS = [greetings_1, greetings_2, greetings_3, greetings_4, greetings_5, greetings_6, greetings_7]

farewell_1 = 'bye'
farewell_2 = "okay bye"
farewell_3 = 'see you'
farewell_4 = 'thanks bye'
farewell_5 = "Bye"
farewell_6 = 'okay'
farewell_7 = 'okay, thanks'
farewell_8 = 'alright'
farewell_9 = 'thanks'
farewell_10 = 'Okay, thanks'
farewell_11 = 'Okay'
farewell_12 = 'No'
farewell_13 = 'Thanks'

FAREWELL = [farewell_1, farewell_2, farewell_3, farewell_4, farewell_5, farewell_6, farewell_7, farewell_8,farewell_9, farewell_10, farewell_11, farewell_12, farewell_13]

In [ ]:
app = Flask(__name__, template_folder = '/content/templates' )
run_with_ngrok(app)
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0
app.config['TEMPLATES_AUTO_RELOAD'] = True

In [ ]:
@app.route("/", methods=['GET', 'POST'])
def chat():
  return render_template('chatroom1.html')

@app.route("/get")
def get_bot_response():
  global chat_history
  userText = request.args.get('msg')
  print(len(userText))
  response = "I am unable to answer your question at the moment. Please contact emergency if this is an urgent matter."
  if len(userText) <= 1:
    response = 'Hmm your question is too short. Maybe you mistyped your question?'
    chat_history = ''
  if len(userText) > 1:
    if userText.startswith('stats:covid'):
      chat_history = ''
      covid19 = COVID19Py.COVID19(data_source="csbs")
      stats = covid19.getLatest()
      confirmed = stats['confirmed']
      deaths = stats['deaths']
      response = f'Here are the quick stats. \n  Total confirmed: {confirmed}, \n Total deaths: {deaths}'
      
    elif re.search("("+")|(".join(PROFANITY)+")", userText):
      response = 'I am sorry but I really do not want to hear any hate speech from you. Please ask a valid question, or leave'
      chat_history = ''
    elif len(userText) < 15 and any(map(userText.startswith, GREETINGS)):
      chat_history = ''
      response = ["Hi there, how can I help you today?", 'Hey, how can I help?', 'How may I help you today?', "Hello there!", "Hi, feel free to ask any questions", "How can I help you today?"]
      response = random.choice(response)
      print(response)
    elif len(userText) < 15 and any(map(userText.startswith, FAREWELL)):
      chat_history = ''
      response = ["Take care and bye!", "Okay bye!", "Thanks for visiting", "Okay stay safe!", "Goodbye!", "Farewell!"]
      response = random.choice(response) 
      print(response) 
    elif len(userText) < 15 and userText.startswith('Yes'):
      chat_history = ''
      response = 'Video functionality not available, try again later'
    else:
      chat_history += ' '+ userText
      response = model.predict([chat_history])[0]      
      print(response)
    response = emoji.emojize(':pill: {}'.format(response))
    
  return response
 
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5e1fff329b41.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/May/2021 13:53:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2021 13:53:36] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [22/May/2021 13:53:37] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/May/2021 13:53:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2021 13:54:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2021 13:54:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2021 13:54:35] "GET /static/style.css HTTP/1.1" 404 -


12


...

127.0.0.1 - - [22/May/2021 13:56:26] "GET /get?msg=Yes HTTP/1.1" 200 -


3
6


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

127.0.0.1 - - [22/May/2021 13:56:35] "GET /get?msg=Ok%20bye HTTP/1.1" 200 -


Break. Ok bye. Let me know if I can assist you further.  



In [ ]:
a = 'Hello'

In [ ]:
a.lower()

'hello'

In [ ]:
model.predict

<bound method Seq2SeqModel.predict of <simpletransformers.seq2seq.seq2seq_model.Seq2SeqModel object at 0x7fa530606090>>

In [ ]:
chat_history = ''
global chat_history